# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment,ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,normal,choice
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.resource_configuration import ResourceConfiguration
import os
import joblib
import requests
import json
import sklearn


In [ ]:
ws = Workspace.from_config()

experiment_name = 'capstone_project_hyperdrive'

experiment=Experiment(ws, experiment_name)


In [ ]:
cpu_cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

## Prepare Data

In [ ]:
remote_dataset = TabularDatasetFactory.from_delimited_files("https://www.openml.org/data/get_csv/16826755/phpMYEkMl")
ds = remote_dataset.to_pandas_dataframe()

if "data" not in os.listdir():
    os.mkdir("./data")

if not os.path.isfile("data/titanic.csv"):
    ds.to_csv('data/titanic.csv',index = False)

## Hyperdrive Configuration

In this expiremnt we are using HyperDrive which helping us to cover a range of hyperparameters to find the best combination of parameteres to acheive the goal which in our case is Maximizing the Accuracy

to detirmine the hyperparametrs what we need to pass to the model and the range of values to cover we are using RandomParameterSampling, which takes the max number of iteration(--max_iter) as a chice of enumeration and the Regularization Strength (--c) as a value between .1 and 1

Another argument that we pass to the hyperdriveconfig is the stopping policy, we are using BanditPolicy, in our case each run which is less than 95% of the best performing run will be terminted, this will eliminate runs that get rsults we don't need.

There is the main argument which is the estimator which is your algorithm that you will apply, we are using SKLearn, this estimator takes the train.py which is the script file that contains your custome code.

The custome code in the train.py using the sklearn LogisticRegression and a method for cleaning the the data, splitting the data to training and testing set.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.05, evaluation_interval=3)  

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
        "--max_iter": choice(50, 75),
        "--C": uniform(.4, .6)
       
    })

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory= './',entry_script='train.py',
compute_target = compute_target)

hyperdrive_run_config = HyperDriveConfig(estimator = estimator,
                                hyperparameter_sampling=param_sampling,
                                policy=early_termination_policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=12,
                                max_concurrent_runs = 4     
                                )

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config,show_output=True)

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [ ]:
hyperdrive_best_run

In [ ]:
hyperdrive_best_run.download_file('outputs/model.pkl', output_file_path='./outputs/model.pkl')

In [ ]:
model = Model.register(workspace=ws,
                       model_name='capstone_hyperdrive_best_model',          
                       model_path='./outputs/model.pkl', 
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version=sklearn.__version__)

## Web Service Deployment

In [ ]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,
                                                       auth_enabled=True,
                                                       enable_app_insights=True)

service_name = 'hyperdrivewebservice'
service = Model.deploy(ws, service_name, [model],deployment_config=deployment_config)
service.wait_for_deployment(True)

### Test the deployed service

In [ ]:
data = {
  "data": [
    {
                  "pclass": 1,
                  "sex": 1,
                  "age": 20,
                  "sibsp": 1,
                  "parch": 1,
                  "embarked": 2
    }
  ],
  "method": "predict"
}

In [ ]:
input_data = json.dumps(data)

primaryKey, secondaryKey = service.get_keys()

scoring_uri = service.scoring_uri

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {primaryKey}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data,headers=headers)

print(resp.json())

## Clear Resources

In [ ]:
# Delete the web service
service.delete()

In [ ]:
# Delete Compute Resources

try:
    compute_target.delete()
except ComputeTargetException:
    print("cpu-cluster Not Found")